## Tutorial on how to use Parcels on 3D C grids

Parcels enables to interpolate field data discretised on a C grid.
C grids are very popular in OGCMs such that velocity fields resulting from those models are often provided on such grids, except if they have been firstly re-interpolated on a A grid.

More information about C-grid interpolation can be found in [Delandmeter et al., 2019](https://www.geosci-model-dev-discuss.net/gmd-2018-339/).
An example of such a discretisation is the model NEMO, which is largely used by Parcels. A tutorial teaching how to [interpolate 2D data on a NEMO grid](https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_nemo_curvilinear.ipynb) is available within Parcels.

Here, we focus on 3D fields. Basically, it is a straightforward extension of the 2D example, but it is very easy to do a mistake in the setup of the vertical discretisation that would affect the interpolation scheme.

### Preliminary comments

*How to know if your data is discretised on a C grid?* The best way is to read the documentation coming along with the data. An easy check is also assess the coordinates of the U, V and W fields: for an A grid, U, V and W are distributed on the same nodes, such that the coordinates are the same. For a C grid, there is a shift of half a cell between the different variables.

*What about grid indexing?* Since the C-grid variables are not located on the same nodes, there is not one obvious way to define the indexing, i.e. where is `u[k,j,i]` compared to `v[k,j,i]` and `w[k,j,i]`. In Parcels, we use the same notation as in NEMO: see [horizontal indexing](https://www.nemo-ocean.eu/doc/img360.png) and [vertical indexing](https://www.nemo-ocean.eu/doc/img362.png).
It is important that you check if your data is following the same notation. Otherwise, you should re-index your data properly (this can be done within Parcels, there is no need to regenerate new netcdf files).


In [1]:
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4_3D
from glob import glob
import numpy as np
from datetime import timedelta as delta
from os import path

# data_path = path.join(path.dirname(__file__), 'NemoCurvilinear_data/')
data_dir = '/Users/delandmeter/data/NEMO-MEDUSA/ORCA025-N006/'
ufiles = sorted(glob(data_dir+'means/ORCA025-N06_200001??d05U.nc'))
vfiles = sorted(glob(data_dir+'means/ORCA025-N06_200001??d05V.nc'))
wfiles = sorted(glob(data_dir+'means/ORCA025-N06_200001??d05W.nc'))
mesh_mask = data_dir + 'domain/coordinates.nc'

filenames = {'U': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': ufiles},
             'V': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': vfiles},
             'W': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': wfiles}}

variables = {'U': 'uo',
             'V': 'vo',
             'W': 'wo'}
dimensions = {'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'}}

fieldset = FieldSet.from_nemo(filenames, variables, dimensions)
fieldset.U.vmax = 5
fieldset.V.vmax = 5
fieldset.W.vmax = 5

# meshgrid containing 5x5 points uniformly distributed in a [0,1]x[0,1] quad
vec = np.linspace(0, 1, 5)
xsi, eta = np.meshgrid(vec, vec)

# get particles in Rhine estuary
lonCorners = [2.96824026, 3.22713804, 3.26175451, 3.002671]
latCorners = [51.60693741, 51.58454132, 51.73711395, 51.759758]
lon_r = (1-xsi)*(1-eta) * lonCorners[0] + xsi*(1-eta) * lonCorners[1] + \
        xsi*eta * lonCorners[2] + (1-xsi)*eta * lonCorners[3]
lat_r = (1-xsi)*(1-eta) * latCorners[0] + xsi*(1-eta) * latCorners[1] + \
        xsi*eta * latCorners[2] + (1-xsi)*eta * latCorners[3]

lonp = lon_r.flatten()
latp = lat_r.flatten()
depthp = np.ones(latp.shape)

pset = ParticleSet.from_list(fieldset, JITParticle, lon=lonp, lat=latp, depth=depthp)
kernels = pset.Kernel(AdvectionRK4_3D)
pset.execute(kernels, runtime=delta(days=15), dt=delta(hours=6))

for p in pset:
    print(p)


         It will be opened with no decoding. Filling values might be wrongly parsed.
INFO: Compiled JITParticleAdvectionRK4_3D ==> /var/folders/h0/01fvrmn11qb62yjw7v1kn62r0000gq/T/parcels-503/8fadc5f90ac9c8f394335371ca1f401a.so
100% (1296000.0 of 1296000.0) |##########| Elapsed Time: 0:00:12 Time:  0:00:12


P[0](lon=3.695555, lat=51.971939, depth=0.467862, time=1296000.000000)
P[1](lon=3.828294, lat=52.013317, depth=0.350941, time=1296000.000000)
P[2](lon=3.930110, lat=52.028423, depth=0.340018, time=1296000.000000)
P[3](lon=3.968106, lat=52.033142, depth=0.286998, time=1296000.000000)
P[4](lon=4.013868, lat=52.008968, depth=0.272774, time=1296000.000000)
P[5](lon=3.664367, lat=51.980835, depth=0.580458, time=1296000.000000)
P[6](lon=3.775796, lat=52.016319, depth=0.373643, time=1296000.000000)
P[7](lon=3.920011, lat=52.047493, depth=0.357648, time=1296000.000000)
P[8](lon=3.960804, lat=52.066895, depth=0.271179, time=1296000.000000)
P[9](lon=4.019048, lat=52.068531, depth=0.244559, time=1296000.000000)
P[10](lon=3.658056, lat=52.010090, depth=0.679452, time=1296000.000000)
P[11](lon=3.761564, lat=52.022896, depth=0.422852, time=1296000.000000)
P[12](lon=3.885614, lat=52.059116, depth=0.365956, time=1296000.000000)
P[13](lon=3.955304, lat=52.086990, depth=0.280503, time=1296000.000000)
P[